In [12]:

%matplotlib auto
## TODO:
# refactorize to make it model -view- presenter
# pythonize slicing based on return value of spanSelector
# make xLabel of axis2 readable always
# add more info to show
# plot same day rectangles differently!
# update, linewidth, selectionColor on axis2
# adjust colors
# rotate labels,
# fix labelformat
# adjust spacing
# generalize button layout

Using matplotlib backend: Qt5Agg


todo

add logic checkers

In [13]:
import os
import inspect
import datetime
from datetime import date, timedelta
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.widgets import Button, SpanSelector
from matplotlib.dates import num2date

In [14]:
class otpParser():
    
    def __init__(self):
        self.headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'currentValue', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'noIdea5']
        
    def parseFiles(self, files):
        dataFrames = [pd.read_csv(file, header=None) for file in files]
        mergedFrame = pd.concat(dataFrames)
        mergedFrame.columns = self.headers                                        
        mergedFrame = mergedFrame.reset_index(drop=True)
        self.mergedFrame = mergedFrame.sort_values(by='date')
        #return self.mergedFrame.head()

    
    def calculateAttributes(self):
        
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
        transactions = {'expense':self.expense,'income':self.income }
        return self.timeAxis, transactions

    def printDf(self):
        return (self.mergedFrame.head())
    
    def calculateDates(self):
        
        start = self.mergedFrame['date'].values[0]
        end = self.mergedFrame['date'].values[-1]
        self.pandasStartDate = "{0}-{1}-{2}".format(str(start)[0:4], str(start)[4:6], str(start)[6:8])
        self.startDate = date(int(str(start)[0:4]), int(str(start)[4:6]), int(str(start)[6:8]))
        self.endDate = date(int(str(end)[0:4]), int(str(end)[4:6]), int(str(end)[6:8]))
        self.timeDelta = self.endDate - self.startDate
        #print (self.startDate, self.endDate, self.timeDelta)
        
    def separateTransactions(self):


        days = [int((self.startDate + timedelta(days=i)).strftime('%Y%m%d')) for i in range(self.timeDelta.days + 1)]
        transDays = self.mergedFrame['date'].values
        heights = self.mergedFrame['sum'].values
        expense = [-figure if figure<0 else 0 for figure in heights]
        income = [figure if figure>0 else 0 for figure in heights]
        Xs = transDays

        values, counts = np.unique(Xs, return_counts=True)
        WholeY=[]
        smallY=np.zeros_like(days)  
        payment = np.zeros_like(days)

        for freq in range(1,max(counts)+1): 
            for val, cnt in zip(values, counts):
                if cnt >= freq:
                    index = np.where(Xs==val)[0][freq-1]
                    if heights[index] > 0:
                        payment [days.index(val)] = heights[index]
                    else:
        #             print (days.index(val), index, val)
                        smallY[days.index(val)] = -heights[index] # maybe transDays
            WholeY.append(smallY)
            smallY=np.zeros_like(days)  


        self.expense = WholeY
        self.income = payment

        pass
        
    def selectMoney(self):
        history = self.mergedFrame['sum']
        
    def createFigures(self):
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
        self.viewer = financeViewer(self.timeAxis,self.charges, self.salary)


In [15]:
class Presenter():
    
    def __init__(self, model, view, files):
        self.model = model
        self.view = view
        self.files = files
        
        self.model.parseFiles(files)
        #self.model.mergeDataFrames()
        self.timeAxis, self.transactions = self.model.calculateAttributes()

#         self.view.connect2presenter(self)
    def showPlots(self):
        self.view.showPlots(self.timeAxis, self.transactions)
    
    def showDataframe(self):
        return self.model.mergedFrame
    
    def onResetClicked(self):
        pass
    
    def onSelect(self):
        pass

In [27]:
RED = '#d62728'
GREY = '#abb2b9'
GREEN = '#196f3d'
WHITE = ''
BLACK = ''
PURPLE = '#aaace2' # Color of the buttons, titles
LIGHT_GREEN = '#a9dfbf'
LIGHT_RED = '#ff9494'
LIGHT_GREY = '#d5d8dc'
width = 0.3
WIDTH = 12
G_RAT = (1 + 5 ** 0.5) / 2 # golden ratio
LABEL_ROTATION = 15 # DEGREES
DATEFORMATSTRING = '%Y-%m-%d'
DATEFROMAT = mdates.DateFormatter(DATEFORMATSTRING)


class financeViewer():
    
    def __init__(self):

        self.box = dict(facecolor='blue', pad=3, alpha=0.2, boxstyle="Round4,pad=0.3")
        self.testString ="""Date: {}
                            Sum: {} HUF
                            Comment: {}
                            yCoord: {}"""
    
    def connect2presenter(self, presenterObject):
        self.presenter = presenterObject
    
    def createFigure(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig = plt.figure(figsize=(WIDTH, WIDTH/G_RAT),facecolor = LIGHT_GREY)

        self.gsp = gridspec.GridSpec(
                nrows = 3, ncols = 2, wspace = 0.05, hspace = 0.45,
                width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

        self.ax1 = plt.subplot(self.gsp[0,:])
        self.ax2 = plt.subplot(self.gsp[1:,0])
        self.ax3 = plt.subplot(self.gsp[1,1])
        self.ax4 = plt.subplot(self.gsp[2,1])
        

    def drawAxes(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        for ax in [self.ax1,self.ax2,self.ax3, self.ax4]: 
            ax.set_facecolor(GREY)
        #####BIG PLOT##
        self.plotAx2()
        self.plotAx1()
        ####ZOOM PLOT##



        ##info plot##
        self.txt = self.ax3.text(0.1,0.5,'',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=12, color='black',
        wrap = True)
        self.ax3.set_xticks([]) 
        self.ax3.set_yticks([]) 
        self.ax3.set_title('info about the transactions', bbox=self.box)

        ### place of buttons##
        self.ax4.set_xticks([]) 
        self.ax4.set_yticks([]) 

    def on_plot_hover(self, event):
#         print ('{} function is called'.format(inspect.stack()[0][3]))
        if not event.inaxes: return
        if event.inaxes!= self.ax2: return

        for idx,bar in enumerate(self.ax2.patches):
            if bar.get_x() < event.xdata < bar.get_x() + bar.get_width():
                if bar.get_y() < event.ydata < bar.get_y() + bar.get_height(): 
                    self.ax2.patches[idx].set_edgecolor(LIGHT_RED)
                    date_ordinal, y = self.ax2.transData.inverted().transform([event.x, event.y])
    
                    # convert the numeric date into a datetime
                    transDate = num2date(date_ordinal).strftime(DATEFORMATSTRING)
                    pdDate = num2date(date_ordinal).strftime('%Y%m%d')
                    print (int(pdDate),bar.get_height() )
#                     comment = df.loc[(df['date'] == int(pdDate)) & (abs(df['sum'],)==bar.get_height()),'comment'].iloc[0]

            
            
                    newStr = self.testString.format(transDate,int(bar.get_height()), 'comment',event.y, )
                    self.txt.set_text(newStr)

            else: 
                self.ax2.patches[idx].set_edgecolor(self.ax2.patches[idx].get_facecolor())
            
        self.fig.canvas.draw()

    def reset_button_on_clicked(self, mouse_event):
        
        pass

        ## redraw axis 2
#         self.ax2.cla()
#         self.ax2.set_title('Selected duration', bbox=self.box)
#         self.ax2.bar(self.timeAxis, self.expense, width, color=RED)
#         self.ax2.bar(self.timeAxis, self.income, width, color=GREEN)
#         plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
#         self.ax2.xaxis.set_major_formatter(DATEFROMAT)
#         self.fig.canvas.draw()

    def balanceView_button_on_clicked(self, mouse_event):
        #print ('{} function is not implemented'.format(inspect.stack()[0][3]))
        pass
    def transView_button_on_clicked(self, mouse_event):
        #print ('{} function is not implemented'.format(inspect.stack()[0][3]))
        pass
    def plotAx2(self, startDate=None, endDate=None):  
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        
        if StartDate:
            zoomedTime = self.timeAxis[startDate:endDate]
            zoomedExpense = [expense[startDate:endDate] for expense in self.expense]
            zoomedIncome = self.income[startDate:endDate]
        else:
            zoomedTime = self.timeAxis
            zoomedExpense = self.expense
            zoomedIncome = self.income
        self.ax2.bar(zoomedTime, zoomedIncome, color=GREEN,edgecolor='black')
        previousBars=copy.deepcopy(zoomedIncome)
        for expense in self.expense:
            self.ax2.bar(self.timeAxis, expense, bottom=previousBars,color=RED, edgecolor='black')
            previousBars+=expense
    def plotAx1(self):
        self.ax1.set_title('Whole duration',bbox=self.box)

        self.ax1.bar(self.timeAxis, self.income, color=GREEN,edgecolor='black')
        previousBars=copy.deepcopy(self.income)
        for expense in self.expense:
            self.ax1.bar(self.timeAxis,expense, bottom=previousBars,color=RED, edgecolor='black')
            previousBars+=expense
#         self.ax1.bar(self.timeAxis, self.expense, color=RED)
        
#         label = self.ax1.set_xlabel('Xlabel', fontsize = 9)
#         self.ax1.xaxis.set_label_coords(1.05, -0.025)
        self.span = SpanSelector(self.ax1, self.onselect, 'horizontal', 
                   rectprops=dict(alpha=0.3, facecolor=RED))

    def onselect(self, xmin, xmax):

        dayMin, dayMax = sorted((int(xmin-0.5), int(xmax+0.5)))
        ##xmin, xmax is days from zero, if Xaxis is pandas daterange

        yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
        startDate = yearZero + timedelta(days=dayMin)
        endDate = yearZero + timedelta(days=dayMax)
        st=str(startDate)[:10]
        nd=str(endDate)[:10]
        stIdx, = np.where( self.timeAxis.values==np.datetime64(st) )
        endIdx, = np.where( self.timeAxis.values==np.datetime64(nd) )
        stIdx , endIdx = stIdx[0], endIdx[0]
#         zoomedTime = self.timeAxis[stIdx:endIdx]
#         zoomedCharges = self.expense[stIdx:endIdx]
#         zoomedSalary = self.income[stIdx:endIdx]
        self.plotAx2(stIdx, endIdx)

#         self.ax2.set_title('Selected duration', bbox=self.box)
#         self.ax2.bar(zoomedTime, zoomedCharges, width, color=RED)
#         self.ax2.bar(zoomedTime, zoomedSalary , width, color=GREEN)
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        self.ax2.xaxis.set_major_locator(DayLocator())

        self.fig.canvas.draw()
        
    def makeButtons(self):

        pos = self.ax4.get_position() # get the  position of axis ,which contains the buttons 
        rowNr, colNr = 1,3
        buttonwidth = 0.1
        buttonheight = 0.07
        Vspace = (pos.width - colNr*buttonwidth)/(colNr+1)
        Hspace = (pos.height - rowNr*buttonheight)/(rowNr+1)
        
        reset_button_ax = self.fig.add_axes([pos.x0+Vspace, pos.y0+Hspace, buttonwidth, buttonheight])
        transactionViewButon_ax = self.fig.add_axes([pos.x0+2*Vspace+buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])
        balanceViewButton_ax = self.fig.add_axes([pos.x0+3*Vspace+2*buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])

        self.reset_button = Button(reset_button_ax, 'Reset', color=PURPLE, hovercolor=RED)
        self.transactionViewButton = Button(transactionViewButon_ax, 'Transactions', color=PURPLE, hovercolor=RED)
        self.balanceViewButton = Button(balanceViewButton_ax, 'Balance', color=PURPLE, hovercolor=RED)
    
    def connectButtons(self):

        self.reset_button.on_clicked(self.reset_button_on_clicked)
        self.transactionViewButton.on_clicked(self.transView_button_on_clicked)
        self.balanceViewButton.on_clicked(self.balanceView_button_on_clicked)

    
    def showPlots(self, timeAxis, transactions):
        self.timeAxis = timeAxis
        self.income = transactions['income']
        self.expense = transactions['expense']
        self.createFigure()
        self.drawAxes()

        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig.canvas.mpl_connect('motion_notify_event', self.on_plot_hover) 
        self.fig.subplots_adjust(left=0.06, bottom=0.07, right=0.97, top=0.95)
        self.makeButtons()
        self.connectButtons()
        plt.setp( self.ax1.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax1.xaxis.set_major_formatter(DATEFROMAT)
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        plt.show()

In [28]:
# myApp.timeAxis
# myApp.transactions

viewer = financeViewer()
viewer.showPlots(myApp.timeAxis ,myApp.transactions)


createFigure function is called
drawAxes function is called
showPlots function is called


In [21]:
print(myApp.timeAxis)
print (myApp.transactions)

DatetimeIndex(['2017-05-01', '2017-05-02', '2017-05-03', '2017-05-04',
               '2017-05-05', '2017-05-06', '2017-05-07', '2017-05-08',
               '2017-05-09', '2017-05-10', '2017-05-11', '2017-05-12',
               '2017-05-13', '2017-05-14', '2017-05-15', '2017-05-16',
               '2017-05-17', '2017-05-18', '2017-05-19', '2017-05-20',
               '2017-05-21', '2017-05-22', '2017-05-23', '2017-05-24',
               '2017-05-25', '2017-05-26', '2017-05-27', '2017-05-28',
               '2017-05-29', '2017-05-30', '2017-05-31', '2017-06-01',
               '2017-06-02', '2017-06-03', '2017-06-04', '2017-06-05',
               '2017-06-06', '2017-06-07', '2017-06-08', '2017-06-09',
               '2017-06-10', '2017-06-11', '2017-06-12', '2017-06-13',
               '2017-06-14', '2017-06-15', '2017-06-16', '2017-06-17',
               '2017-06-18', '2017-06-19', '2017-06-20', '2017-06-21',
               '2017-06-22', '2017-06-23', '2017-06-24', '2017-06-25',
      

In [ ]:
viewer.

In [20]:
files =  [file for file in os.listdir() if file.lower().endswith('csv')]
model = otpParser()
view = financeViewer()


myApp = Presenter(model, view, files)

In [9]:
myApp.showPlots()

createFigure function is called
drawAxes function is called
showPlots function is called


In [ ]:
a=myApp.showDataframe()
df=a

In [ ]:
a.head(2)

In [ ]:
start = df['date'].values[0]
end = df['date'].values[-1]
pandasStartDate = "{0}-{1}-{2}".format(str(start)[0:4], str(start)[4:6], str(start)[6:8])
startDate = date(int(str(start)[0:4]), int(str(start)[4:6]), int(str(start)[6:8]))
endDate = date(int(str(end)[0:4]), int(str(end)[4:6]), int(str(end)[6:8]))
timeDelta = endDate - startDate

days = [int((startDate + timedelta(days=i)).strftime('%Y%m%d')) for i in range(timeDelta.days + 1)]
transDays = df['date'].values
heights = df['sum'].values
expense = [-figure if figure<0 else 0 for figure in heights]
income = [figure if figure>0 else 0 for figure in heights]
Xs = transDays

values, counts = np.unique(Xs, return_counts=True)
WholeY=[]
smallY=np.zeros_like(days)  
payment = np.zeros_like(days)

for freq in range(1,max(counts)+1): 
    for val, cnt in zip(values, counts):
        if cnt >= freq:
            index = np.where(Xs==val)[0][freq-1]
            if heights[index] > 0:
                payment [days.index(val)] = heights[index]
            else:
#             print (days.index(val), index, val)
                smallY[days.index(val)] = -heights[index] # maybe transDays
    WholeY.append(smallY)
    smallY=np.zeros_like(days)  

    
fig, ax = plt.subplots()
## stack them on each other automatically, create init bottom:
# previousBars=np.zeros_like(smallY)
previousBars=ax.bar(np.arange(len(smallY)),payment, color='green', edgecolor='black')
previousBars=payment
for smallY in WholeY:
    currentBars=ax.bar(np.arange(len(smallY)),smallY, bottom=previousBars,color='red', edgecolor='black')
    previousBars+=smallY
    

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.DataFrame({'x': np.arange(25), 'y': np.random.normal(0,2500,25)})
fig, ax = plt.subplots()

ax.scatter(df.x, df.y, c=np.sign(df.y), cmap="bwr")

plt.show()

In [ ]:
print (values, counts, max(counts))
print (transDays, len(transDays))
print (days, len(days))
print (heights, len(heights))

In [ ]:
Xs=np.array([0,1,1,1,3,4,4,6,6,6,7,8,9])
heights = np.array([10,9,8,5,7,6,4,3,2,1,1,12,1])
values, counts = np.unique(ys, return_counts=True)
print (values, counts, max(counts))

In [ ]:
WholeY=[]
smallY=np.zeros(max(ys)+1)  


In [ ]:

for freq in range(1,max(counts)+1): 

    for val, cnt in zip(values, counts):
        
        if cnt >= freq:

            index = np.where(ys==val)[0][freq-1]
            smallY[val] = heights[index]
    WholeY.append(smallY)
    smallY=np.zeros(max(Xs)+1)  

In [ ]:
WholeY

In [ ]:
fig, ax = plt.subplots()
## stack them on each other automatically, create init bottom:
previousBars=np.zeros_like(smallY)
for smallY in WholeY:
    currentBars=ax.bar(np.arange(len(smallY)),smallY, bottom=previousBars)
    previousBars=smallY
plt.show()

In [ ]:
ax.bar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
Xs=np.array([0,1,1,1,3,4,4,6,6,6,7,8,9])
heights = np.array([10,9,8,5,7,6,4,3,2,1,1,12,1])
values, counts = np.unique(indexes, return_counts=True)
print (values, counts, max(counts))

WholeY=[]
smallY=np.zeros(max(Xs)+1)  

for freq in range(1,max(counts)+1): 
    for val, cnt in zip(values, counts):
        if cnt >= freq:
            index = np.where(ys==val)[0][freq-1]
            smallY[val] = heights[index]
    WholeY.append(smallY)
    smallY=np.zeros(max(Xs)+1) 

fig, ax = plt.subplots()
## stack them on each other automatically, create init bottom:
previousBars=np.zeros_like(smallY)
for smallY in WholeY:
    currentBars=ax.bar(np.arange(len(smallY)),smallY, bottom=previousBars)
    previousBars=smallY
plt.show()

In [ ]:
ls = [(len(df.loc[(df["date"] == datum), "sum"].values),datum ) for datum in np.unique(df['date'].values)]
print (ls)

In [ ]:
ys = [(5,3),3,2,4,6]

In [ ]:
xs = np.arange(len(ys))
xs

In [ ]:
fig, ax = plt.subplots()
ax.bar(xs,ys)

In [ ]:
ax.bar(xs,ys)

In [ ]:
date1 = datetime.datetime(2000, 3, 2)
date2 = datetime.datetime(2000, 3, 6)
delta = datetime.timedelta(hours=6)
dates = drange(date1, date2, delta)

y = arange(len(dates)*1.0)

fig, ax = plt.subplots()
ax.plot_date(dates, y*y)


In [ ]:
print (df.loc[(df['date'] == 20170502) & (abs(df['sum'],)==1080),'comment'].iloc[0])
print (df.loc[(df['date'] == 20170502) & (abs(df['sum'],)==9500),'comment'].iloc[0])


In [ ]:
(df['gender'] == 'male') & (df['pet1'] == df['pet2'] )

In [ ]:
print (a['currency'].mode()[0])

In [ ]:
# View test:
rangeD=22
datAxis = pd.date_range("1986-01-02", periods=rangeD, freq='D')
viewer = financeViewer()
viewer.showPlots(datAxis ,{'expense':np.random.rand(rangeD)*40000,'income':np.random.rand(rangeD)*40000 })

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from numpy import arange

date1 = datetime.datetime(2000, 3, 2)
date2 = datetime.datetime(2000, 3, 6)
delta = datetime.timedelta(hours=6)
dates = drange(date1, date2, delta)

y = arange(len(dates)*1.0)

fig, ax = plt.subplots()
ax.plot_date(dates, y*y)

# this is superfluous, since the autoscaler should get it right, but
# use date2num and num2date to convert between dates and floats if
# you want; both date2num and num2date convert an instance or sequence
ax.set_xlim(dates[0], dates[-1])

# The hour locator takes the hour or sequence of hours you want to
# tick, not the base multiple

ax.xaxis.set_major_locator(DayLocator())
ax.xaxis.set_minor_locator(HourLocator(arange(0, 25, 6)))
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))

ax.fmt_xdata = DateFormatter('%Y-%m-%d %H:%M:%S')
# fig.autofmt_xdate()

plt.show()

In [ ]:
fig, ax = plt.subplots()
datAxis = pd.date_range("1996-01-02", periods=rangeD, freq='D')
ax.plot(datAxis, np.random.rand(rangeD))
def on_move(event):
    ax = event.inaxes
    if ax is not None:
        # convert x y device coordinates to axes data coordinates
        date_ordinal, y = ax.transData.inverted().transform([event.x, event.y])
    
        # convert the numeric date into a datetime
        date = num2date(date_ordinal)
        print (event.x,event.y, date.strftime('%Y'))


on_move_id = fig.canvas.mpl_connect('motion_notify_event', on_move)
lambda pos: mdates.num2date(pos)
#plt.show()

In [ ]:
def on_move(event):
    ax = event.inaxes
    if ax is not None:
        # convert x y device coordinates to axes data coordinates
        date_ordinal, y = ax.transData.inverted().transform([event.x, event.y])

        # convert the numeric date into a datetime
        date = num2date(date_ordinal)

        # sort the quotes by their distance (in time) from the mouse position
        def sorter(quote):
            return abs(quote[0] - date_ordinal)
        quotes.sort(key=sorter)

        print 'on date %s the nearest 3 openings were %s at %s respectively' % \
                        (date, 
                         ', '.join([str(quote[1]) for quote in quotes[:3]]),
                         ', '.join([str(num2date(quote[0])) for quote in quotes[:3]]))


on_move_id = fig.canvas.mpl_connect('motion_notify_event', on_move)
